In [ ]:
from flask import Flask,request,jsonify
import json
import psycopg2
import os
import pandas as pd
from flask_cors import CORS

#import Connector CLass from Features File
from Features import Connector

app=Flask(__name__)
CORS(app)


# Define a route for the root URL("/") of the web application
@app.route("/")
def Home():
    return "API has been successfully created"


# Define a route for GET requests to '/get_employees_data'
@app.route("/get_employees_data", methods=["GET"])  # Define a route for GET requests to '/get_employees_data'
def get_employee_sales_data():
    C_obj = Connector()  # Instantiate the Connector class
    conn = C_obj.connector()  # Establish a connection to the database using the connector method
   
    cur = conn.cursor()  # Create a cursor object for executing queries
    limit = request.args.get('limit', default=0, type=int)  # Get the 'limit' parameter from the query string, default to 0 if not provided
    offset = request.args.get('offset', default=0, type=int)  # Get the 'offset' parameter from the query string, default to 0 if not provided
   
    try:
        query = '''
        SELECT
            e1.employeeNumber,
            e1.lastName,
            e1.firstName,
            e1.extension,
            e1.email,
            e1.officeCode,
            e1.jobTitle,
            o.city,
            o.phone,
            e1.reportsTo,
            e2.lastName AS reportToLastName,
            e2.firstName AS reportToFirstName
        FROM
            employees e1
        JOIN
            offices o ON e1.officeCode = o.officeCode
        LEFT JOIN
            employees e2 ON e1.reportsTo = e2.employeeNumber
        '''  # Define the SQL query to fetch employee and office data with a self-join for managers
        
        params = []  # Initialize an empty list to hold query parameters
        if limit > 0:  # Check if 'limit' parameter is greater than 0
            query += ' LIMIT %s'  # Append the LIMIT clause to the query
            params.append(limit)  # Add the 'limit' value to the params list
        if offset > 0:  # Check if 'offset' parameter is greater than 0
            query += ' OFFSET %s'  # Append the OFFSET clause to the query
            params.append(offset)  # Add the 'offset' value to the params list
        
 
        cur.execute(query, tuple(params))  # Execute the query with the parameters tuple
        
        table_data = cur.fetchall()  # Fetch all the results from the executed query
       
        # Convert the fetched data to a list of dictionaries
        columns = [desc[0] for desc in cur.description]  # Get column names from the cursor description
        json_data = [dict(zip(columns, row)) for row in table_data]  # Combine column names with each row to create dictionaries
        
        return jsonify(json_data), 200  # Return the data as a JSON response with status code 200
   
    except Exception as e:  # Handle any exceptions that occur
        conn.rollback()  # Rollback the transaction in case of an error
        return jsonify({"error": str(e)}), 500  # Return the error message as a JSON response with status code 500
    finally:
        cur.close()  # Close the cursor
        conn.close()  # Close the database connection

if __name__=='__main__':
    app.run(host='127.0.0.1',port=9008,debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9008
 * Running on http://127.0.0.1:9008
Press CTRL+C to quitPress CTRL+C to quit

127.0.0.1 - - [28/Jun/2024 20:25:16] "GET /get_employees_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2024 20:25:16] "GET /get_employees_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2024 20:25:16] "GET /get_employees_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2024 20:25:16] "GET /get_employees_data HTTP/1.1" 200 -
